In [1]:
# Initial imports.
import pandas as pd
import numpy as np
from numpy import random
import os
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Import RDS password
from config import rds_pwd

import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.9 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-26 00:27:41--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-05-26 00:27:42 (11.5 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Wine-Project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Configure settings for RDS
mode = "append"
#jdbc_url="jdbc:postgresql://<connection string>:5432/<database-name>"
jdbc_url="jdbc:postgresql://wine-final-project.czqkltznl3rl.us-east-2.rds.amazonaws.com/winedb"
#config = {"user":"postgres",
          #"password": "<password>",
          #"driver":"org.postgresql.Driver"}
config = {"user":"wineuser",
          "password": rds_pwd,
          "driver":"org.postgresql.Driver"}

In [5]:
wineweather = pd.read_csv('/content/weather_scaled_updated.csv', index_col=0)
wineweather.head(4)

,winery,country,points,variety,winery_search,latitude,longitude,Average Temperature (Kelvin),Average Air Pressure (hPa),Average Humidity (%),Average Daily Precipitation (mm)
0,St. Julian,US,87,Riesling,St. Julian winery US,42.212251,-85.891713,283.486181,1016.253874,73.468022,0.118022
1,Sweet Cheeks,US,87,Pinot Noir,Sweet Cheeks winery US,43.956670,-123.279135,284.928462,1016.972170,75.501813,0.108269
2,Kirkland Signature,US,87,Cabernet Sauvignon,Kirkland Signature winery US,35.896713,-84.140546,288.217995,1017.277418,74.106346,0.129918
3,Louis M. Martini,US,87,Cabernet Sauvignon,Louis M. Martini winery US,38.491708,-122.452308,287.733599,1015.947830,74.851236,0.076868


In [6]:
wineweather.dtypes

winery                               object
country                              object
points                                int64
variety                              object
winery_search                        object
latitude                            float64
longitude                           float64
Average Temperature (Kelvin)        float64
Average Air Pressure (hPa)          float64
Average Humidity (%)                float64
Average Daily Precipitation (mm)    float64
dtype: object

In [8]:
allwines = wineweather.drop(columns=['winery'])
uswinedata = wineweather[wineweather['country'] == 'US'].drop(columns=['winery','country'])
allwines.head(2)

,country,points,variety,winery_search,latitude,longitude,Average Temperature (Kelvin),Average Air Pressure (hPa),Average Humidity (%),Average Daily Precipitation (mm)
0,US,87,Riesling,St. Julian winery US,42.212251,-85.891713,283.486181,1016.253874,73.468022,0.118022
1,US,87,Pinot Noir,Sweet Cheeks winery US,43.956670,-123.279135,284.928462,1016.972170,75.501813,0.108269


In [9]:
allwines.columns

Index(['country', 'points', 'variety', 'winery_search', 'latitude',
       'longitude', 'Average Temperature (Kelvin)',
       'Average Air Pressure (hPa)', 'Average Humidity (%)',
       'Average Daily Precipitation (mm)'],
      dtype='object')

In [10]:
county_weath = pd.read_csv('/content/uscounty_weather_v2.csv', index_col=0).dropna()
county_weath.head(2)

,zip,lat,lng,city,state_id,state_name,county_name,Average Temperature (Kelvin),Average Air Pressure (hPa),Average Humidity (%),Average Daily Precipitation (mm)
0,1001,42.06259,-72.62589,Agawam,MA,Massachusetts,Hampden,283.257967,1016.12956,70.273709,0.126154
1,1002,42.37492,-72.46210,Amherst,MA,Massachusetts,Hampshire,283.257967,1016.12956,70.273709,0.126154


In [11]:
county_weath.dtypes

zip                                   int64
lat                                 float64
lng                                 float64
city                                 object
state_id                             object
state_name                           object
county_name                          object
Average Temperature (Kelvin)        float64
Average Air Pressure (hPa)          float64
Average Humidity (%)                float64
Average Daily Precipitation (mm)    float64
dtype: object

In [12]:
len(county_weath)

1796

In [13]:
county_weath.columns

Index(['zip', 'lat', 'lng', 'city', 'state_id', 'state_name', 'county_name',
       'Average Temperature (Kelvin)', 'Average Air Pressure (hPa)',
       'Average Humidity (%)', 'Average Daily Precipitation (mm)'],
      dtype='object')

In [14]:
county_ids = county_weath.drop(columns=['lat','lng','Average Temperature (Kelvin)', 'Average Air Pressure (hPa)',
       'Average Humidity (%)', 'Average Daily Precipitation (mm)'])
county_ids.head(2)

,zip,city,state_id,state_name,county_name
0,1001,Agawam,MA,Massachusetts,Hampden
1,1002,Amherst,MA,Massachusetts,Hampshire


In [15]:
bigcounty = county_weath.drop(columns=['city', 'state_id', 'state_name', 'county_name'])
bigcounty.head(2)

,zip,lat,lng,Average Temperature (Kelvin),Average Air Pressure (hPa),Average Humidity (%),Average Daily Precipitation (mm)
0,1001,42.06259,-72.62589,283.257967,1016.12956,70.273709,0.126154
1,1002,42.37492,-72.46210,283.257967,1016.12956,70.273709,0.126154


In [16]:
bigcounty.dtypes

zip                                   int64
lat                                 float64
lng                                 float64
Average Temperature (Kelvin)        float64
Average Air Pressure (hPa)          float64
Average Humidity (%)                float64
Average Daily Precipitation (mm)    float64
dtype: object

In [17]:
countydrops = ['zip']

In [18]:
mer_df = allwines[allwines['variety'] == 'Merlot']
syr_df = allwines[allwines['variety'] == 'Syrah']
savyb_df = allwines[allwines['variety'] == 'Sauvignon Blanc']
ries_df = allwines[allwines['variety'] == 'Riesling']
cabsav_df = allwines[allwines['variety'] == 'Cabernet Sauvignon']
char_df = allwines[allwines['variety'] == 'Chardonnay']
pino_df = allwines[allwines['variety'] == 'Pinot Noir']

In [20]:
winedfs = [mer_df,syr_df,savyb_df,ries_df,cabsav_df,char_df,pino_df]
for x in winedfs:
  print(len(x))

2983
3944
4729
5110
8837
11334
12681


In [21]:
mer_df.dtypes

country                              object
points                                int64
variety                              object
winery_search                        object
latitude                            float64
longitude                           float64
Average Temperature (Kelvin)        float64
Average Air Pressure (hPa)          float64
Average Humidity (%)                float64
Average Daily Precipitation (mm)    float64
dtype: object

In [22]:
mer_df.columns

Index(['country', 'points', 'variety', 'winery_search', 'latitude',
       'longitude', 'Average Temperature (Kelvin)',
       'Average Air Pressure (hPa)', 'Average Humidity (%)',
       'Average Daily Precipitation (mm)'],
      dtype='object')

In [25]:
winedropcols = ['country', 'points', 'variety', 'winery_search']

In [28]:
#features and targets
variety = mer_df.copy()
y = variety['points']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestRegressor(n_estimators=200, random_state=99)
rf.fit(X, y)
rf.score(X,y)

0.672360307056741

In [29]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
mer_preds = rf.predict(Xcount)

In [30]:
#features and targets
variety = syr_df
y = variety['points']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestRegressor(n_estimators=200, random_state=99)
rf.fit(X, y)
rf.score(X,y)

0.6524464002909762

In [31]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
syr_preds = rf.predict(Xcount)

In [32]:
#features and targets
variety = savyb_df
y = variety['points']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestRegressor(n_estimators=200, random_state=99)
rf.fit(X, y)
rf.score(X,y)

0.606807418587975

In [33]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
savyb_preds = rf.predict(Xcount)

In [34]:
#features and targets
variety = ries_df
y = variety['points']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestRegressor(n_estimators=200, random_state=99)
rf.fit(X, y)
rf.score(X,y)

0.5776133570979387

In [35]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
ries_preds = rf.predict(Xcount)

In [36]:
#features and targets
variety = cabsav_df
y = variety['points']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestRegressor(n_estimators=200, random_state=99)
rf.fit(X, y)
rf.score(X,y)

0.6294553517631403

In [37]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
cabsav_preds = rf.predict(Xcount)

In [38]:
#features and targets
variety = char_df
y = variety['points']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestRegressor(n_estimators=200, random_state=99)
rf.fit(X, y)
rf.score(X,y)

0.6299301452245211

In [39]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
char_preds = rf.predict(Xcount)

In [40]:
#features and targets
variety = pino_df
y = variety['points']
X = variety.drop(columns=winedropcols)
X_train, X_test, y_train, y_test = train_test_split(X,y)
#init model again
rf = RandomForestRegressor(n_estimators=200, random_state=99)
rf.fit(X, y)
rf.score(X,y)

0.5568064612261503

In [41]:
#using model to predict the counties
Xcount = bigcounty.drop(columns=countydrops)
pino_preds = rf.predict(Xcount)

In [68]:
mer_preds

array([86.79353968, 87.33325496, 86.6014929 , ..., 87.5567399 ,
       87.93357738, 87.96945365])

In [76]:
counties_link = bigcounty.copy()
counties_link['mer_points'] = mer_preds
counties_link['syr_points'] = syr_preds
counties_link['savyb_points'] = savyb_preds
counties_link['ries_points'] = ries_preds
counties_link['cabsav_points'] = cabsav_preds
counties_link['char_points'] = char_preds
counties_link['pinot_points'] = pino_preds
counties_predsorg = counties_link.merge(county_ids, how='inner', on='zip')
counties_predsorg.head(5)

,zip,lat,lng,Average Temperature (Kelvin),Average Air Pressure (hPa),Average Humidity (%),Average Daily Precipitation (mm),mer_points,syr_points,savyb_points,ries_points,cabsav_points,char_points,pinot_points,city,state_id,state_name,county_name
0,1001,42.06259,-72.62589,283.257967,1016.129560,70.273709,0.126154,86.793540,86.394583,84.610405,87.139362,84.656649,86.251520,89.590490,Agawam,MA,Massachusetts,Hampden
1,1002,42.37492,-72.46210,283.257967,1016.129560,70.273709,0.126154,87.333255,85.738500,85.196927,88.036993,84.577375,86.496111,90.434877,Amherst,MA,Massachusetts,Hampshire
2,1005,42.42017,-72.10615,283.368324,1015.384533,69.594835,0.135027,86.601493,86.313833,83.942952,87.257696,85.773633,86.611280,88.517383,Barre,MA,Massachusetts,Worcester
3,1029,42.19632,-73.04836,283.257967,1016.129560,70.273709,0.126154,86.760220,86.414500,84.589381,87.053820,84.431565,85.984005,89.377765,East Otis,MA,Massachusetts,Berkshire
4,1054,42.47363,-72.48746,283.257967,1016.129560,70.273709,0.126154,87.372503,85.733000,84.869548,87.046141,84.180292,86.486824,90.454710,Leverett,MA,Massachusetts,Franklin


In [78]:
savyb_df['points'].describe()

count    4729.000000
mean       87.456122
std         2.679951
min        80.000000
25%        86.000000
50%        87.000000
75%        89.000000
max        96.000000
Name: points, dtype: float64

In [116]:
counties_linkrnd = bigcounty.copy()
counties_linkrnd['mer_points'] = np.rint(mer_preds)
counties_linkrnd['syr_points'] = np.rint(syr_preds)
counties_linkrnd['savyb_points'] = np.rint(savyb_preds)
counties_linkrnd['ries_points'] = np.rint(ries_preds)
counties_linkrnd['cabsav_points'] = np.rint(cabsav_preds)
counties_linkrnd['char_points'] = np.rint(char_preds)
counties_linkrnd['pinot_points'] = np.rint(pino_preds)
counties_predsrnd = counties_linkrnd.merge(county_ids, how='inner', on='zip')
counties_predsrnd.head(5)

,zip,lat,lng,Average Temperature (Kelvin),Average Air Pressure (hPa),Average Humidity (%),Average Daily Precipitation (mm),mer_points,syr_points,savyb_points,ries_points,cabsav_points,char_points,pinot_points,city,state_id,state_name,county_name
0,1001,42.06259,-72.62589,283.257967,1016.129560,70.273709,0.126154,87.0,86.0,85.0,87.0,85.0,86.0,90.0,Agawam,MA,Massachusetts,Hampden
1,1002,42.37492,-72.46210,283.257967,1016.129560,70.273709,0.126154,87.0,86.0,85.0,88.0,85.0,86.0,90.0,Amherst,MA,Massachusetts,Hampshire
2,1005,42.42017,-72.10615,283.368324,1015.384533,69.594835,0.135027,87.0,86.0,84.0,87.0,86.0,87.0,89.0,Barre,MA,Massachusetts,Worcester
3,1029,42.19632,-73.04836,283.257967,1016.129560,70.273709,0.126154,87.0,86.0,85.0,87.0,84.0,86.0,89.0,East Otis,MA,Massachusetts,Berkshire
4,1054,42.47363,-72.48746,283.257967,1016.129560,70.273709,0.126154,87.0,86.0,85.0,87.0,84.0,86.0,90.0,Leverett,MA,Massachusetts,Franklin


In [119]:
counties_predsorg.to_csv('counties_logisticpreds_org.csv')
counties_predsorg.to_json('counties_logisticpreds_org.json')
counties_predsrnd.to_csv('counties_logisticpreds_rnd.csv')
counties_predsrnd.to_json('counties_logisticpreds_rnd.json')

In [120]:
machinecols = [ 'mer_points', 'syr_points',
       'savyb_points', 'ries_points', 'cabsav_points', 'char_points',
       'pinot_points']
for x in machinecols:
  print(counties_predsrnd[x].value_counts())

86.0    544
85.0    445
87.0    334
88.0    159
84.0    134
89.0    112
82.0     33
83.0     31
90.0      4
Name: mer_points, dtype: int64
87.0    453
88.0    433
89.0    332
86.0    223
90.0    131
85.0    113
91.0     67
84.0     22
83.0     16
92.0      6
Name: syr_points, dtype: int64
86.0    520
87.0    518
88.0    311
85.0    303
84.0     69
89.0     39
83.0     14
90.0     13
91.0      9
Name: savyb_points, dtype: int64
87.0    738
86.0    429
88.0    254
85.0    190
89.0    106
84.0     52
90.0     24
83.0      2
82.0      1
Name: ries_points, dtype: int64
87.0    471
88.0    414
86.0    345
89.0    224
85.0    158
90.0     88
84.0     56
91.0     20
83.0      7
82.0      7
93.0      4
92.0      2
Name: cabsav_points, dtype: int64
87.0    481
86.0    398
85.0    290
88.0    287
89.0    149
84.0    101
90.0     31
83.0     30
91.0     21
82.0      7
92.0      1
Name: char_points, dtype: int64
87.0    522
88.0    454
86.0    286
89.0    192
90.0    136
85.0     87
91.0     73
84.

In [46]:
#counties_preds.to_csv('county_predictions.csv')
#counties_preds.to_json('county_predictions.json')

In [47]:
#spark_counties = spark.createDataFrame(counties_preds)
#spark_counties.show()

In [48]:
#spark_counties.write.jdbc(url=jdbc_url, table='county_predictions', mode=mode, properties=config)

In [49]:
#sparked_preds = spark.read.jdbc(url=jdbc_url, table='county_predictions', properties=config)

In [50]:
#sparked_preds.show()

In [51]:
#winners_df = pd.DataFrame(sparked_preds.select('zip','lat','lng','state_id','county_name','mer_success','rose_success','syr_success','savyb_success','ries_success','cabsav_success','char_success','pinot_success').collect(), columns=['zip','lat','lng','state','county_name','mer_success','rose_success','syr_success','savyb_success','ries_success','cabsav_success','char_success','pinot_success'], dtype=float)
#winners_df.describe()

In [52]:
#winners_df.columns

In [53]:
#winners_df

In [54]:
#winners_df['mer_success'].value_counts()